# CSV processing for import into airtable perennial and vendor fields

See [reference doc](./perennials-louisiana-iris.md).

## Clean up from resarch on Louisiana irises August 2025

Found Claude and Gemini both struggled to keep the csv clean on edits, filling empty values with speculative values.

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../../temp/louisiana-irises.csv')


In [3]:
# Basic derived fields - had copiled the species level name into the column earlier
df["Scientific name"] = df["Scientific name"].astype(str) + " " + df["Name"].astype(str)

df["Name"] = df["Name"].astype(str) + " Louisiana iris"

In [4]:
# Clean up fields 

# Replace empty fields with placeholder "-"
for col in ["Vigor", 'Color class',"Standards", "Falls", "Features","Awards"]:
    df[col] = df[col].fillna('-')
# Replace empty fields with empty string ""
for col in ["Additional growth"]:
    df[col] =df[col].fillna('')
    
# Ensure columns with strings ends with a period, except for '-' or ''
for col in ["Features", "Vigor", "Additional growth" ]:
    df[col] = df[col].astype(str).apply(lambda x: x if x in ['-', ''] else (x if x.endswith(".") else x + "."))
# Ensure column strings begin with a capital letter.
for col in ["Vigor", "Additional growth" ]:
    df[col] = df[col].astype(str).apply(lambda x: x[0].upper() + x[1:] if len(x)>0 else x)
for col in ["Standards", "Falls", "Features"]:
    df[col] = df[col].astype(str).apply(lambda x: x[0].lower() + x[1:] if len(x)>0 else x)


In [5]:
# generate Note string from research fields

# Full population:
# {Vigor} {Additional growth}. [{Hybridizer}. R. {Release year}.] Color class-{'Color class'} 
#   Standards: {Standards}; falls: {Falls}; features: {Features}. Awards: {Awards}

# In "clean up fields" section above, 
# * empty "Vigor" and 'Color class' are populated with "-"
# * empty "Additional growth" is populated with ""


# Hybridizer and release year string missing data adaptions
# * if either is missing, only show the one that exists.
# * if both are missing, Note contains placeholder "[-]"

df["Hybridizer bracket"] = df.apply(lambda row: f"{row['Hybridizer']}. R. {row['Release year']}." if pd.notna(row['Hybridizer']) and pd.notna(row['Release year']) else (f"{row['Hybridizer']}." if pd.notna(row['Hybridizer']) else (f"R. {row['Release year']}." if pd.notna(row['Release year']) else "-")), axis=1)


df["temp"]=df["Vigor"].astype(str)+ " "+ df["Additional growth"].astype(str) + " [" + df["Hybridizer bracket"].astype(str) + "] "
df["temp"]=df["temp"].astype(str)+" Color class-"+ df["Color class"].astype(str) + " "
df["temp"]=df["temp"].astype(str)+" Standards: "+ df["Standards"].astype(str) +"; falls: "+ df["Falls"].astype(str) +"; features: "+ df["Features"].astype(str)
df["temp"]=df["temp"].astype(str)+" Awards: "+ df["Awards"].astype(str) 

df["import note"] = df["temp"]

In [6]:
df["Code"] = "Interesting"
df["Plan code"] = "Iris import"
df["Use"] = "landscaping, rain garden"
df["Origin"] = "US - South"

In [7]:
df.columns

Index(['Name', 'Bloom time', 'Vigor', 'Scientific name', 'Genetics',
       'Additional growth', 'Hybridizer', 'Release year', 'Color class',
       'Standards', 'Falls', 'Features', 'Awards', 'Header', 'Flower',
       'height (ft)', 'price', 'size', 'Vendor', 'Hybridizer bracket', 'temp',
       'import note', 'Code', 'Plan code', 'Use', 'Origin'],
      dtype='object')

In [8]:
# generate final dataframes to export to temp/perennial.csv and temp/vendor-perennial.csv
perennialDF = df[["Name",  "Code", "Plan code", "Genetics", "Scientific name", "Header", 
                  "height (ft)","Bloom time","Flower",'Use','Origin',"import note"]]
vendorDF = df[["Name","Vendor",'price', 'size']]
vendorDF.columns = ["Perennials","Vendor",'price', 'size']
perennialDF.to_csv('../../temp/perennial.csv', index=False)
vendorDF.to_csv('../../temp/vendor-perennial.csv', index=False) 